# News

### Мы решили торговать на бирже, причем так, чтобы решение о покупке акций принимала нейросеть на основе последних новостей о той или иной компании. Для этого нужно научиться классифицировать все новости. Дана база новостей из разных источников news.csv. Необходимо написать модель, классифицирующую новости по источникам.

In [2]:
import numpy as np 
import pandas as pd
import torch
import transformers 
from tqdm import notebook 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('../input/news-opendb/news (1).csv')
display(data.head(10))

In [4]:
data.info()
data['source'].value_counts()
data.isnull().sum()

In [5]:
data_drop = data.dropna()
data_drop = data_drop.head(12000)
data_drop.isnull().sum()

In [6]:
tokenizer = transformers.BertTokenizer(
    vocab_file='../input/deeppavlov-rubertbasecased/vocab.txt')

### Сначала сделаем предсказание для данных, где источник новостей известен, тем самым натренируем модель.

In [7]:
cut_data = data_drop['news'].apply(lambda x: x[:512])
print(cut_data)

In [8]:
#Чтобы токенизировать все новости, применим метод apply():
tokenized = cut_data.apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

print(tokenized)

#Применим метод padding, чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT. Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. 
#Остальные векторы дополним нулями.
#Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется attention. Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения

max_len = 0
for i in tokenized.values:
     if len(i) > max_len:
        max_len = len(i)
print(max_len)        
#tokenized_values = tokenized.values[:, :512]
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [9]:
print(tokenized.values)

In [10]:
print(padded.shape)

In [11]:
#Инициализируем конфигурацию BertConfig (англ. Bert Configuration). 
#В качестве аргумента передадим ей JSON-файл с описанием настроек модели. 
config = transformers.BertConfig.from_json_file(
    '../input/deeppavlov-rubertbasecased/config.json')
model = transformers.BertModel.from_pretrained(
    '../input/deeppavlov-rubertbasecased/pytorch_model.bin', config=config) 

In [12]:
print(attention_mask.shape)

Начнём преобразование текстов в эмбеддинги.
Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим (например, 100).

In [13]:
batch_size = 100 
# сделаем пустой список для хранения эмбеддингов твитов
embeddings = []
# Сделаем цикл по батчам. Отображать прогресс будет функция notebook():
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    # преобразуем данные в формат тензоров (англ. tensor) — многомерных векторов в библиотеке torch
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    # преобразуем маску
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
# Для ускорения вычисления функцией no_grad() в библиотеке torch укажем, 
# что градиенты не нужны: модель BERT обучать не будем.
    with torch.no_grad():
        # Чтобы получить эмбеддинги для батча, передадим модели данные и маску:
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
# Из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов:
# преобразуем элементы методом numpy() к типу numpy.array
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

Обучим модель логистической регрессии на эмбеддингах.
Для корректного тестирования поделим их на обучающую и тестовую выборки в соотношении 50:50.

In [15]:
#Соберём все эмбеддинги в матрицу признаков вызовов функции concatenate():
features = np.concatenate(embeddings)
target = data_drop['source']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)

In [16]:
#print(features_train.shape)
#print(target_train.shape)
#print(features_test.shape)
#print(features_test.head(10))
#print(target_test.shape)
#print(target_test.head(10))

In [18]:
# Логистическая регрессия
model_regression = LogisticRegression(random_state=12345, solver='liblinear') # инициализируем модель LogisticRegression
model_regression.fit(features_train, target_train) # обучаем модель на тренировочной выборке
result_regression = model_regression.score(features_test, target_test) # считаем качество модели на тестовой выборке
predictions = model_regression.predict(features_test)
print(result_regression)
print(predictions)

### Сделаем предсказание для данных, где источник новостей неизвестен, с помощью ранее натренированной модели.

In [ ]:
data_for_predict = data[data['source'].isnull()]
print(data_for_predict)

In [ ]:
cut_data_for_predict = data_for_predict['news'].apply(lambda x: x[:512])
print(cut_data_for_predict)

In [ ]:
tokenized = cut_data_for_predict.apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

print(tokenized)

max_len = 0
for i in tokenized.values:
     if len(i) > max_len:
        max_len = len(i)
print(max_len)        

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

print(padded.shape)
print(attention_mask.shape)

In [ ]:
batch_size = 100 

embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
      
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [ ]:
features_new = np.concatenate(embeddings)

result = model_regression.predict(features_new)

print(result)

In [ ]:
pd.DataFrame(result).to_csv("predict.csv", index = None)